In [8]:
#Dependencies
import pandas as pd
import requests
import json

In [ ]:
# Get user input for the data that the user wants to research
#Nationwide or Statewide
data_input = "Nationwide"



In [ ]:
historic_US_values_URL = "https://api.covidtracking.com/v1/us/daily.json"
statewide_URL = base_url = f"https://api.covidtracking.com/v1/states/{state}/{date}.json"

In [ ]:
if data_input == "Nationwide":
    print('User chose Nationwide')
    base_url = historic_US_values_URL
elif data_input == "statewide":
    print('User chose statewide')
    base_url = statewide_URL

In [10]:
state = "ca"
date = "20201001"


In [11]:
#Make a test API Call 


response = requests.get(base_url).json()

print(json.dumps(response))

{"date": 20201001, "state": "CA", "positive": 813687, "probableCases": null, "negative": 13958164, "pending": null, "totalTestResultsSource": "totalTestsViral", "totalTestResults": 14771851, "hospitalizedCurrently": 3205, "hospitalizedCumulative": null, "inIcuCurrently": 817, "inIcuCumulative": null, "onVentilatorCurrently": null, "onVentilatorCumulative": null, "recovered": null, "dataQualityGrade": "B", "lastUpdateEt": "10/1/2020 02:59", "dateModified": "2020-10-01T02:59:00Z", "checkTimeEt": "09/30 22:59", "death": 15888, "hospitalized": null, "dateChecked": "2020-10-01T02:59:00Z", "totalTestsViral": 14771851, "positiveTestsViral": null, "negativeTestsViral": null, "positiveCasesViral": 813687, "deathConfirmed": null, "deathProbable": null, "totalTestEncountersViral": null, "totalTestsPeopleViral": null, "totalTestsAntibody": null, "positiveTestsAntibody": null, "negativeTestsAntibody": null, "totalTestsPeopleAntibody": null, "positiveTestsPeopleAntibody": null, "negativeTestsPeopleA